# Obtención de datos de la BBDD

In [ ]:
import polars as pl
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from dotenv import load_dotenv
import os




In [ ]:
# Creación de la clase pero con .env y Polars, teniendo que cambiar el código original

class DatabaseConnection:
    def __init__(self):
        load_dotenv()  # Variables de entorno
        port = 3306  # Puerto por defecto de MySQL
        user = os.getenv("DB_USER")
        password = os.getenv("DB_PASSWORD")
        host = os.getenv("DB_HOST")
        nombre_base_datos = os.getenv("DB_NAME") 
        self.engine = create_engine(
            f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{nombre_base_datos}"
        )

    def query_to_polars(self, sql_query):
        try:
            with self.engine.connect() as conn:
                result = conn.execute(text(sql_query))
                columns = result.keys()
                rows = result.fetchall()
                if not rows:
                    print("Consulta ejecutada correctamente pero no hay resultados.")
                    return pl.DataFrame(schema=columns)
                print("Consulta ejecutada correctamente.")
                return pl.DataFrame(rows, schema=columns)
        except SQLAlchemyError as e:
            print(f"Error de SQLAlchemy al ejecutar la consulta:\{e}")
            return None
        except Exception as e:
            print(f"Error inesperado:\{e}")
            return None

## Conexión y consulta a la base de datos

In [ ]:
# Usamos y probamos la clase
db = DatabaseConnection()

query = """
SELECT *
FROM ventas_diarias_estudio
WHERE ARTICULO = '3960'
"""

ventas = db.query_to_polars(query)
ventas

## Nota sobre los cambios respecto al notebook original

En esta versión del proyecto, he realizado varios cambios respecto al código base proporcionado:

- **Uso de Polars en vez de pandas:**  
  He optado por seguir utilizando Polars para la manipulación de datos en Python, ya que es una librería más eficiente y rápida para grandes volúmenes de datos, además de ofrecer una sintaxis moderna y muy potente para análisis y transformaciones.

- **Conexión a la base de datos con SQLAlchemy:**  
  En lugar de conectores más simples como `mysql-connector` o `pymysql`, sigo utilizando SQLAlchemy, que es más flexible y profesional, y permite una mejor gestión de conexiones, transacciones y portabilidad del código.

- **Gestión de credenciales con dotenv:**  
  Para evitar exponer contraseñas y parámetros sensibles en el código, he usado la librería `python-dotenv` y guardo los datos de conexión en un archivo `.env`, que está excluido del repositorio con `.gitignore` para mayor seguridad.

- **Manejo de errores y código más robusto:**  
  Se ha añadido control de errores y mensajes informativos en las funciones de consulta para asegurar que cualquier fallo en la conexión o en las queries sea fácil de identificar y depurar, acción que ayuda mucho para identificar errores de todo tipo.

En definitiva, estos cambios son para seguir un poco la lógica del primer notebook, y seguir usando las mismas librerías y dependencias.
